
<div style="text-align: center; line-height: 0; padding-top: 9px;">  
  <img src="/Volumes/cmidev/default/preventech/DBX_hack/compnovai_logo.PNG" alt="Image">  
</div> 

# Fine-Tuning a Pre-trained LLM Model

This notebook performs fine tuning of a pre-trained LLM model called "Bert_Base_Uncased" available on HuggingFace.

Below steps are performed during Fine-Tuning and evaluation:

 - Prepare necessary training and testing data for fine-tuning embedding model 
 - Fine-tune a popular embedding model 
- Evaluate the model using accuracy metrics on validation set of the data
 - Log and serve the trained model on MLFlow

In this notebook, we will fine-tunes Hugging Face Model provided by google called "https://huggingface.co/google-bert/bert-base-uncased" by downloading  model artifacts and uploading those to a volume location as our databricks environment does not allow direct access to the hugging face models.

In [0]:
%pip install mlflow==2.14.1 sentence-transformers==3.0.1 torch==2.3.0 transformers==4.40.1 databricks-sdk==0.31.1 tf-keras==2.16.0 accelerate==0.27.2
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.5/545.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.5 MB/s eta 0:00:00
     ━━━━

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from random import randint
from typing import List, Callable

from databricks.sdk import WorkspaceClient
from databricks.sdk.errors import ResourceDoesNotExist, ResourceAlreadyExists
from databricks.sdk.service.vectorsearch import VectorIndexType, DeltaSyncVectorIndexSpecResponse, EmbeddingSourceColumn, PipelineType
from databricks.sdk.service.serving import ServedEntityInput, EndpointCoreConfigInput

from langchain_core.embeddings import Embeddings

from mlflow.tracking.client import MlflowClient

from datasets import Dataset 

import sentence_transformers, requests, time, json, mlflow, yaml, os, torch
from sentence_transformers import InputExample, losses
from sentence_transformers.losses import CachedMultipleNegativesRankingLoss
from sentence_transformers import SentenceTransformer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
from transformers import AutoTokenizer

from torch.utils.data import DataLoader
import torch

import tempfile

from databricks.sdk.service.serving import AutoCaptureConfigInput

2024-10-23 02:52:18.020319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 02:52:18.037755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 02:52:18.037812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 02:52:18.050065: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 02:52:18.965695: W tensorflow/compiler/tf

In [0]:
%sh PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True # Prevent PyTorch from hogging too much GPU memory in reserve.

In [0]:

VS_ENDPOINT = "vs_endpoint_ccims_fallback"

gpu_is_available = torch.cuda.is_available()

In [0]:
gpu_is_available

True

In [0]:
VS_INDEX_FULL_NAME= f"cmidev.cbu_ccims_poc.claim_text_embeddings_index"


# Concatenate text of selected features

In [0]:
def ConcatenateFields(test_claims):
    from pyspark.sql.functions import concat_ws, struct
    
    df_final_updated = test_claims.withColumn("FC", concat_ws(",", "FC_1","FC_2","FC_3","FC_4","FC_5"))
    final_test_df = df_final_updated.withColumn("DOCUMENT", struct("FC", "ENGINE_NAME_DESC", "FAILCODE",  "CLEANED_CORRECTION"))\
    .drop("FC",'FC 1', 'FC 2', 'FC 3', 'FC 4', 'FC 5')

    return final_test_df

In [0]:
# def get_label(claim_id):
#   label = spark.sql("select FAILURE_MODE_BUCKET from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where CLAIM_ID_SEQ =  "+str(claim_id)+" ").toPandas()
#   return (label['FAILURE_MODE_BUCKET'].iloc[0])


# Prepare Training Data

In [0]:
def CreateDatasetWithoutFailModeBucket(limit):
    query = "select * from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where FAILURE_MODE_BUCKET != 'Not Inspected' limit "+str(limit)+""
    test_claims = spark.sql(query)

    final_test_df=ConcatenateFields(test_claims)
    final_test_df = final_test_df.toPandas()
    

    return final_test_df
df = CreateDatasetWithoutFailModeBucket(20000)
df = df[["CLAIM_ID_SEQ","DOCUMENT","FAILURE_MODE_BUCKET"]]
df.rename(columns={"CLAIM_ID_SEQ":"claim_id",
              "DOCUMENT":"document",
              "FAILURE_MODE_BUCKET":"label"},inplace=True)

df.head(4)

,claim_id,document,label
0,23418225,"{'FC': '3716.0,5177.0,1897.0', 'ENGINE_NAME_DE...",ACT(E) ELECTROLYTIC C26 FAILURE
1,16442048,"{'FC': '687.0', 'ENGINE_NAME_DESC': 'X15 3 201...",TSS - CORROSION IN SENSOR BODY
2,16951708,"{'FC': '3361.0,3389.0,3382.0,5584.0,2198.0', '...",ACT(E) CORROSION OF HOUSING AT CONNECTOR
3,16795651,"{'FC': '5943.0', 'ENGINE_NAME_DESC': 'X15 1 20...",THRUST BEARING WORN


In [0]:
len(np.unique(df['label']))
display(df.groupby('label').count())

claim_id,document
6,6
1,1
1,1
560,560
9,9
103,103
639,639
4,4
3,3
430,430


# Identify singleton classes and oversample them

In [0]:
#Identify singleton classes having single example in df
import pandas as pd  
from imblearn.over_sampling import RandomOverSampler ,SMOTE 
  
  
# Get the embeddings and labels  
embeddings = df['document'].tolist()  
labels = df['label'].tolist()  
  
# Compute the class counts    
class_counts = pd.Series(labels).value_counts()    
    
# Find the singleton classes    
singleton_classes = class_counts[class_counts == 1].index    
    
# Filter the dataframe to remove samples with singleton classes    
mask = ~pd.Series(labels).isin(singleton_classes)    
df_filtered = df[mask]  
  
# Perform oversampling on the singleton classes  
mask = pd.Series(labels).isin(singleton_classes)    
df_filtered_single = df[mask]  
# display(df_filtered_single)
X = (df_filtered_single['document'].tolist())
Y = (df_filtered_single['label'].tolist())

oversampler = RandomOverSampler()  

# Repeat the singleton class examples  
df_repeated = pd.concat([df[df['label'] == c].sample(class_counts.max(), replace=True, random_state=42) for c in singleton_classes])  
  
 
# display(df_repeated)
  
# Combine the resampled data with the filtered data  
df =  pd.concat([df_filtered,df_repeated],ignore_index=True, axis=0)

In [0]:
display(df.groupby('label').count())

claim_id,document
6,6
4214,4214
4214,4214
560,560
9,9
103,103
639,639
4,4
3,3
430,430


# Perform train/test split

In [0]:
from sklearn.model_selection import train_test_split  
import numpy as np  
labels = df['label'].tolist()
document = df['document'].tolist()
X_train, X_test, y_train, y_test = train_test_split(document, labels, test_size=0.05, random_state=42, shuffle=True, stratify=labels)  


In [0]:
len(X_train)

79035

# Training Labels

In [0]:
print((np.unique(y_train)))

['ACT(E) ADJUSTED/DAMAGED BY CUSTOMER' 'ACT(E) BOBBIN COIL SHORT'
 'ACT(E) BROKEN SHAFT' 'ACT(E) BROKEN SOLDER JOINT'
 'ACT(E) CASTING DEFECT' 'ACT(E) CBA COMPONENT/PROCESS DEFECT'
 'ACT(E) CONSEQUENTIAL DAMAGE' 'ACT(E) CONTAMINATION OF CRYSTAL'
 'ACT(E) COOLING BODY POROSITY' 'ACT(E) CORROSION OF HOUSING AT CONNECTOR'
 'ACT(E) DAMAGE INDUCED (ELECTRICAL OVER STRESS)'
 'ACT(E) DAMAGE INDUCED (IMPACTS)'
 'ACT(E) DAMAGE INDUCED (THERMAL OVERSTRESS)'
 'ACT(E) DAMAGED CONNECTOR SEAL' 'ACT(E) ELECTROLYTIC C26 FAILURE'
 'ACT(E) GEAR TRAIN WEAR' 'ACT(E) HANDLING/TRANSIT DAMAGE'
 'ACT(E) I-GEAR FAILURE' 'ACT(E) INNER LAYER PCB SHORT'
 'ACT(E) LOOSE APS MAGNET' 'ACT(E) LOOSE MOTOR MAGNET'
 'ACT(E) MICROPROCESSOR ISSUE' 'ACT(E) NON-HOLSET PARTS'
 'ACT(E) OUTPUT GEAR AXIAL PLAY' 'ACT(E) OUTPUT GEAR MAGNET BROKEN/LOOSE'
 'ACT(E) SOLDER ISSUE' 'ACT(E) SUPPLIER ANALYSIS NOT CONDUCTED'
 'ACT(E) SUPPLIER MANUFACTURING FAULT' 'ACT(E) SYSTEMS INTEGRATION ISSUE'
 'ACT(E) TROUBLE NOT FOUND' 'ACT(E) UNKNOW

# Perform encoding of labels and convert encoded labels to PyTorch Tensors

In [0]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch

# Assuming y_train and y_test are pandas Series or lists of string labels

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the label encoder and transform y_train and y_test to integer labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the encoded labels to PyTorch tensors
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)


In [0]:
len(y_train_encoded)

79035

In [0]:
# y_test_encoded

array([23,  4,  4,  4,  4])

# Preprocess text in X_train and X_test

In [0]:
selected_keys = ['FC', 'ENGINE_NAME_DESC', 'FAILCODE', 'CLEANED_CORRECTION']  # Example keys you want to keep
X_train_selected = [{key: x[key] for key in selected_keys if key in x} for x in X_train]
X_test_selected = [{key: x[key] for key in selected_keys if key in x} for x in X_test]

In [0]:
X_train_strings = [str(x).lower() for x in X_train_selected]
X_test_strings = [str(x).lower() for x in X_test_selected]

# Create Tokenizer of Pre-trained Model

In [0]:
#Create a tokenizer object of Google's Bert Base Uncased Model and tokenize training data. Here code files are downloaded from Hugging Face and uploaded to the mentioned volume location.
from transformers import AutoTokenizer
bert_model_path = "/Volumes/cmidev/default/preventech/DBX_hack/google-bertbert-base-uncased"
# Initialize the tokenizer (example with BERT)
tokenizer = AutoTokenizer.from_pretrained(bert_model_path)

# Function to tokenize and encode the text data in the dictionaries
def encode_texts(text_dicts, tokenizer):
    #Use only claim narrative
    # texts = [text_dict['CLEANED_CORRECTION'] for text_dict in text_dicts]  # Extract text data

    #Use all concatenated features
    texts = text_dicts
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Tokenize and encode X_train and X_test
#For claim narrative only, use below:
# X_train_encoded = encode_texts(X_train, tokenizer)
# X_test_encoded = encode_texts(X_test, tokenizer)
#For more feaures use below:
X_train_encoded = encode_texts(X_train_strings, tokenizer)
X_test_encoded = encode_texts(X_test_strings, tokenizer)

# Perform training of the pre-trained model for fine tuning on custom dataset

In [0]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments,TFBertForSequenceClassification
import torch
import numpy as np
# Assuming you have a binary classification task
num_labels = len(np.unique(y_train))  # Adjust based on your task

# Load the pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(bert_model_path, num_labels=num_labels)

# Custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Assuming y_train_encoded is your list/array of labels
train_dataset = CustomDataset(X_train_encoded, y_train_encoded)
val_dataset = CustomDataset(X_test_encoded, y_test_encoded)
# Define training arguments
output_dir ='/Volumes/cmidev/default/preventech/DBX_hack/model_training_checkpoint'
training_args = TrainingArguments(
    output_dir=output_dir,          # Output directory
    num_train_epochs=100,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir=output_dir,            # Directory for storing logs
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset         

)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /Volumes/cmidev/default/preventech/DBX_hack/google-bertbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/.ipykernel/2244/command-3735586181635829-1436925236:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,2.316800
1000,0.709900
1500,0.707600
2000,0.651100
2500,0.622500
3000,0.554700
3500,0.546000
4000,0.533300
4500,0.543100
5000,0.559800


/root/.ipykernel/2244/command-3735586181635829-1436925236:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/root/.ipykernel/2244/command-3735586181635829-1436925236:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/root/.ipykernel/2244/command-3735586181635829-1436925236:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.i

TrainOutput(global_step=988000, training_loss=1.8580542503387822, metrics={'train_runtime': 268143.4362, 'train_samples_per_second': 29.475, 'train_steps_per_second': 3.685, 'total_flos': 2.0810666055208796e+18, 'train_loss': 1.8580542503387822, 'epoch': 100.0})

# Save the model at volume path for later use

In [0]:
# Save the model
# model.save_pretrained('/Volumes/cmidev/default/preventech/DBX_hack/ccims_bert_base_uncased_trained_model_v0')

#Model with limited feaures, but larger dataset 10/13/24, took 3-days to train
model.save_pretrained('/Volumes/cmidev/default/preventech/DBX_hack/ccims_bert_base_uncased_trained_model_v1')


# Single Test Case with CPU Cluster

In [0]:
# from transformers import BertTokenizer

# # Assuming 'model' is your BertForSequenceClassification model
# # and 'tokenizer' is an instance of BertTokenizer or similar compatible with your model

# input_text = "SRT190CQ overlap with 190CLCheck engine light on, Connected to Cummins and inspected codes, Fault code active for turbo speed sensor, Opened eds to troubleshoot, Inspected if any other fault code was triggering turbo speed code, Inspected sensor supply voltage, voltage was in spec, Inspect intake manifold pressure sensor for carbon build up, removed sensor to inspect, no build up found, Inspect for signal wire open circuit, Removed harness from ECM and ohmed signal to signal from sensor connector to ecm connector, That was less than 10 ohms, Inspected return for a open circuit, ohmed from sensor connector to ecm connector and that was also less than 10 ohms, Checked for turbo sensor shorted to ground, It was greater than 100k ohms, Inspected signal pin and ohmed out signal pin at ecm to inspect for a pin to pin short, All other pins on ecm connecter are not shorted to signal pin for turbo speed sensor connector, Checked ecm calibration history, Update is available but not relavent to active code, Advised replace turbo speed senor, Speed sensor ordered, Replaced speed sensor"

# # Tokenize the input
# inputs = tokenizer.encode_plus(
#     input_text,
#     add_special_tokens=True,
#     return_tensors="pt"
# )

# # Get predictions
# with torch.no_grad():
#     outputs = model(**inputs)

# # Check if outputs is a tuple (for older versions of transformers)
# if isinstance(outputs, tuple):
#     logits = outputs[0]
# else:
#     logits = outputs.logits

# # Assuming you want the class with the highest score
# predictions = torch.argmax(logits, dim=-1)

# # Display predictions
# display(predictions)

# Single test case with GPU cluster

In [0]:
from transformers import BertTokenizer
import torch

# Assuming 'model' is your BertForSequenceClassification model
# and 'tokenizer' is an instance of BertTokenizer or similar compatible with your model

input_text = "SRT190CQ overlap with 190CLCheck engine light on, Connected to Cummins and inspected codes, Fault code active for turbo speed sensor, Opened eds to troubleshoot, Inspected if any other fault code was triggering turbo speed code, Inspected sensor supply voltage, voltage was in spec, Inspect intake manifold pressure sensor for carbon build up, removed sensor to inspect, no build up found, Inspect for signal wire open circuit, Removed harness from ECM and ohmed signal to signal from sensor connector to ecm connector, That was less than 10 ohms, Inspected return for a open circuit, ohmed from sensor connector to ecm connector and that was also less than 10 ohms, Checked for turbo sensor shorted to ground, It was greater than 100k ohms, Inspected signal pin and ohmed out signal pin at ecm to inspect for a pin to pin short, All other pins on ecm connecter are not shorted to signal pin for turbo speed sensor connector, Checked ecm calibration history, Update is available but not relavent to active code, Advised replace turbo speed senor, Speed sensor ordered, Replaced speed sensor"

# Tokenize the input
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    return_tensors="pt"
)

# Move the inputs to the same device as the model
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Get predictions
with torch.no_grad():
    outputs = model(**inputs)

# Check if outputs is a tuple (for older versions of transformers)
if isinstance(outputs, tuple):
    logits = outputs[0]
else:
    logits = outputs.logits

# Assuming you want the class with the highest score
predictions = torch.argmax(logits, dim=-1)

# Display predictions
display(predictions)

tensor([14], device='cuda:0')

In [0]:
a = label_encoder.inverse_transform(predictions.cpu())[0]
a

'ACT(E) ELECTROLYTIC C26 FAILURE'

# Batch Testing
Perform prediction of label on fine tuned model on 5000 test cases and evaluate model performance

In [0]:

claims_train = df['claim_id'].tolist()
claims_str = ",".join([f"'{claim}'" for claim in claims_train])


query = f"select * from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where FAILURE_MODE_BUCKET != 'Not Inspected' AND CLAIM_ID_SEQ not  in ({claims_str}) limit 5000"



test_claims =spark.sql(query)

final_test_df=ConcatenateFields(test_claims)
# test_claims_list = test_claims.select("CLEANED_CORRECTION").collect()
docs = final_test_df.select('DOCUMENT').toPandas()['DOCUMENT'].tolist()
test_claims_list = [str(doc).lower() for doc in docs]
final_test_df_pd = final_test_df.toPandas()
# test_claims_list = final_test_df_pd['CLEANED_CORRECTION'].tolist()#").collect()
claim_id = final_test_df_pd["CLAIM_ID_SEQ"].tolist()#.collect()]
actual_label = final_test_df_pd['FAILURE_MODE_BUCKET'].tolist()
predicted_label_fh =[]

# print(test_claims_list)
for input_text, claim_no, actual_l in zip(test_claims_list,claim_id,actual_label):
  print("claim_no:",claim_no)
  # Tokenize the input
  try:
    # input_text = input_text.lower()

    # Truncate the input to the maximum length the model can handle
    max_length = 512
    inputs = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,  # Ensure truncation if the input exceeds max_length
        return_tensors="pt"
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}



    # Get predictions
    with torch.no_grad():
      outputs = model(**inputs)
    # print("output:",outputs)
    # Check if outputs is a tuple (for older versions of transformers)
    if isinstance(outputs, tuple):
      logits = outputs[0]
    else:
      logits = outputs.logits

    # Assuming you want the class with the highest score
    predictions = torch.argmax(logits, dim=-1)

    # Display predictions
    # display(predictions)
    # label_fh = label_encoder.inverse_transform(predictions)
    # predicted_label_fh.append(label_fh[0])

    label_fh = label_encoder.inverse_transform(predictions.cpu())[0]
    predicted_label_fh.append(label_fh)

    # print("processing claim id:",claim_no, "----actual label:",actual_l,"----Prediction:",label_fh[0])
    print("processing claim id:",claim_no, "----actual label:",actual_l,"----Prediction:",label_fh)

  except:
    print("Too long input","processing claim id:",claim_no,)
    predicted_label_fh.append("Too Long Input")
     


claim_no: 18498037
processing claim id: 18498037 ----actual label: ACT(E) LOOSE APS MAGNET ----Prediction: WRONG PART USED
claim_no: 16359731
processing claim id: 16359731 ----actual label: ACT(E) WRONG PART RETURNED ----Prediction: ACT(E) ELECTROLYTIC C26 FAILURE
claim_no: 21968678
processing claim id: 21968678 ----actual label: TSS - WELD PROCESS DEFECT ----Prediction: WRONG PART USED
claim_no: 17242587
processing claim id: 17242587 ----actual label: ACT(E) CONSEQUENTIAL DAMAGE ----Prediction: ACT(E) ELECTROLYTIC C26 FAILURE
claim_no: 23958345
processing claim id: 23958345 ----actual label: TSS - TROUBLE NOT FOUND ----Prediction: ACT(E) ELECTROLYTIC C26 FAILURE
claim_no: 21339556
processing claim id: 21339556 ----actual label: ACT(E) WRONG PART RETURNED ----Prediction: TSS - CONSEQUENTIAL DAMAGE
claim_no: 17760890
processing claim id: 17760890 ----actual label: ACT(E) TROUBLE NOT FOUND ----Prediction: VG CROSS SHAFT SLIP
claim_no: 18016941
processing claim id: 18016941 ----actual lab

In [0]:
import pandas as pd
result_df = pd.DataFrame()
result_df['claim_id'] = claim_id
result_df['actual_label'] = actual_label
result_df['predicted_label_few'] = predicted_label_fh
display(result_df)

claim_id,actual_label,predicted_label_few
18498037,ACT(E) LOOSE APS MAGNET,WRONG PART USED
16359731,ACT(E) WRONG PART RETURNED,ACT(E) ELECTROLYTIC C26 FAILURE
21968678,TSS - WELD PROCESS DEFECT,WRONG PART USED
17242587,ACT(E) CONSEQUENTIAL DAMAGE,ACT(E) ELECTROLYTIC C26 FAILURE
23958345,TSS - TROUBLE NOT FOUND,ACT(E) ELECTROLYTIC C26 FAILURE
21339556,ACT(E) WRONG PART RETURNED,TSS - CONSEQUENTIAL DAMAGE
17760890,ACT(E) TROUBLE NOT FOUND,VG CROSS SHAFT SLIP
18016941,TSS - INTERMITTENT SIGNAL,ACT(E) ELECTROLYTIC C26 FAILURE
23485408,ACT(E) ELECTROLYTIC C26 FAILURE,ACT(E) ELECTROLYTIC C26 FAILURE
18008102,ACT(E) ELECTROLYTIC C26 FAILURE,ACT(E) ELECTROLYTIC C26 FAILURE


In [0]:
#Calculate accuracy pretrained on cleaned correction
correct=0
for actual, predicted in zip(actual_label,predicted_label_fh):
    if actual==predicted:
        correct+=1
print("Accuracy of pretrained model on 3 features (FC, FailCode, Claim Narrative) % =",correct*100/len(actual_label))

Accuracy of pretrained model on 3 features (FC, FailCode, Claim Narrative) % = 9.1


# Log & Serve Model through MLFlow
- Provide input and output example for model signature
- Register model under Databricks Catalog under models as:  'cmidev.default.bert_fine_tuned_20k_v1'.


In [0]:

from transformers import BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained('/Volumes/cmidev/default/preventech/DBX_hack/ccims_bert_base_uncased_trained_model_v1')

In [0]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [0]:
import mlflow
import torch
import io
from mlflow.models.signature import infer_signature
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments,TFBertForSequenceClassification
from transformers import Trainer


# Initialize or import your trainer here
# from your_module import Trainer
trainer = Trainer(model=model)
registered_model_name = "bert_fine_tuned_20k_v1"

input_text = "SRT190CQ overlap with 190CLCheck engine light on, Connected to Cummins and inspected codes, Fault code active for turbo speed sensor, Opened eds to troubleshoot, Inspected if any other fault code was triggering turbo speed code, Inspected sensor supply voltage, voltage was in spec, Inspect intake manifold pressure sensor for carbon build up, removed sensor to inspect, no build up found, Inspect for signal wire open circuit, Removed harness from ECM and ohmed signal to signal from sensor connector to ecm connector, That was less than 10 ohms, Inspected return for a open circuit, ohmed from sensor connector to ecm connector and that was also less than 10 ohms, Checked for turbo sensor shorted to ground, It was greater than 100k ohms, Inspected signal pin and ohmed out signal pin at ecm to inspect for a pin to pin short, All other pins on ecm connecter are not shorted to signal pin for turbo speed sensor connector, Checked ecm calibration history, Update is available but not relavent to active code, Advised replace turbo speed senor, Speed sensor ordered, Replaced speed sensor"
# Use the converted arrays for infer_signature
signature = infer_signature(model_input=input_text, model_output='ACT(E) ELECTROLYTIC C26 FAILURE')

with mlflow.start_run() as run:
    # Load the PyTorch model into a buffer
    buffer = io.BytesIO()
    torch.save(trainer.model.state_dict(), buffer)  # Assuming 'trainer.model' is your model
    buffer.seek(0)
    
    # Log the model to mlflow with the signature
    mlflow.pytorch.log_model(trainer.model, "model", signature=signature)
    
    # Register the model under the specified catalog and model name
    mlflow.register_model(f"runs:/{run.info.run_id}/model", registered_model_name)

/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

Successfully registered model 'bert_fine_tuned_20k_v1'.
2024/10/22 20:53:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: bert_fine_tuned_20k_v1, version 1
Created version '1' of model 'bert_fine_tuned_20k_v1'.


# Test registered model URL

In [0]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch

# Initialize the label encoder
le = LabelEncoder()


y_train=['ACT(E) ADJUSTED/DAMAGED BY CUSTOMER',
         'ACT(E) BOBBIN COIL SHORT',
 'ACT(E) BROKEN SHAFT',
'ACT(E) BROKEN SOLDER JOINT',
'ACT(E) CASTING DEFECT',
'ACT(E) CBA COMPONENT/PROCESS DEFECT',
'ACT(E) CONSEQUENTIAL DAMAGE',
'ACT(E) CONTAMINATION OF CRYSTAL',
 'ACT(E) COOLING BODY POROSITY','ACT(E) CORROSION OF HOUSING AT CONNECTOR',
 'ACT(E) DAMAGE INDUCED (ELECTRICAL OVER STRESS)',
 'ACT(E) DAMAGE INDUCED (IMPACTS)',
 'ACT(E) DAMAGE INDUCED (THERMAL OVERSTRESS)',
 'ACT(E) DAMAGED CONNECTOR SEAL','ACT(E) ELECTROLYTIC C26 FAILURE',
 'ACT(E) GEAR TRAIN WEAR','ACT(E) HANDLING/TRANSIT DAMAGE',
 'ACT(E) I-GEAR FAILURE','ACT(E) INNER LAYER PCB SHORT',
 'ACT(E) LOOSE APS MAGNET','ACT(E) LOOSE MOTOR MAGNET',
 'ACT(E) MICROPROCESSOR ISSUE','ACT(E) NON-HOLSET PARTS',
 'ACT(E) OUTPUT GEAR AXIAL PLAY','ACT(E) OUTPUT GEAR MAGNET BROKEN/LOOSE',
 'ACT(E) SOLDER ISSUE','ACT(E) SUPPLIER ANALYSIS NOT CONDUCTED',
 'ACT(E) SUPPLIER MANUFACTURING FAULT','ACT(E) SYSTEMS INTEGRATION ISSUE',
 'ACT(E) TROUBLE NOT FOUND','ACT(E) UNKNOWN/INDETERMINATE',
 'ACT(E) WAITING SUPPLIER ANALYSIS','ACT(E) WATER CONTAMINATION (788)',
 'ACT(E) WORN INTERNAL GEARS','ACT(E) WRONG PART RETURNED',
 'ADJUSTED/DAMAGED BY CUSTOMER','COMP HSG CRACKED/BROKEN',
 'CONSEQUENTIAL DAMAGE','CROSS SHAFT MANUFACTURING FAULT',
 'CUMMINS MANUFACTURING FAULT','HANDLING/TRANSIT DAMAGE',
 'IMPELLER WHEEL FOD','IMPELLER WHEEL LCF','MAINTENANCE ERROR',
 'NOZZLE RING ASSY RIVET(S) BROKEN/WORN',
 'NOZZLE SEAL RING BINDING/SEIZED (INNER)','NOZZLE VANES BINDING/SEIZED',
 'OIL CONTAMINATION','OIL DELAY/STARVATION','PENDING GROUP REVIEW',
 'PENDING LAB RESPONSE','SHROUD PLATE LEADING EDGE WEAR',
 'SUPPLIER MANUFACTURING FAULT','THRUST BEARING WORN','TROUBLE NOT FOUND',
 'TSS - ADJUSTED/DAMAGED BY CUSTOMER','TSS - BODY CRACKED/BROKEN',
 'TSS - BODY FAILURE','TSS - BODY POROSITY','TSS - BROKEN COIL WIRE',
 'TSS - CABLE CRACKED/BROKEN','TSS - CABLE FRETTING',
 'TSS - CONSEQUENTIAL DAMAGE','TSS - CONTAMINATION OF COIL',
 'TSS - CORROSION IN SENSOR BODY','TSS - DAMAGED BY CUSTOMER',
 'TSS - INTERMITTENT SIGNAL','TSS - MANUFACTURING FAULT',
 'TSS - MELTED HARNESS','TSS - OVERSTRESSED SKEIN WIRE',
 'TSS - SUPPLIER ANALYSIS NOT CONDUCTED','TSS - TROUBLE NOT FOUND',
 'TSS - UNKNOWN/INDETERMINATE','TSS - WAITING SUPPLIER ANALYSIS',
 'TSS - WELD PROCESS DEFECT','TSS - WRONG PART RETURNED',
 'TURBINE HSG CRACKED/BROKEN','TURBINE WHEEL HCF','TURBINE WHEEL LCF',
 'TURBINE/NOZZLE FOD','UNKNOWN/INDETERMINATE','VG CONTROL LINKAGE FAILURE',
 'VG CROSS SHAFT SLIP','VG PUSHROD BROKEN','WRONG PART RETURNED',
 'WRONG PART USED']
print(len(y_train))
 # Fit the label encoder and transform y_train and y_test to integer labels
y_train_encoded= le.fit_transform((y_train))


86


In [0]:
# Create label encoder as a model artifacts for prediction

# Save the LabelEncoder object and any other configuration data as a dictionary  
config = {  
    "label_encoder": label_encoder,  
}  

# Save the dictionary as a config.pkl file in the model artifacts directory  
model_dir = 'dbfs:/databricks/mlflow-tracking/2267125048865302/0bab96b924a5448b95ac7119d2ecb886/artifacts/model'
config_path = os.path.join(model_dir, "data", "config.pkl") 
print(config_path) 
os.makedirs(os.path.dirname(config_path), exist_ok=True)  
with open(config_path, "wb") as f: 
  pickle.dump(config, f)  


dbfs:/databricks/mlflow-tracking/2267125048865302/0bab96b924a5448b95ac7119d2ecb886/artifacts/model/data/config.pkl


In [0]:
import mlflow
from transformers import AutoTokenizer
import os
import pickle
import torch

bert_model_path = "/Volumes/cmidev/default/preventech/DBX_hack/google-bertbert-base-uncased"
# Initialize the tokenizer (example with BERT)
tokenizer = AutoTokenizer.from_pretrained(bert_model_path)

logged_model = 'runs:/0bab96b924a5448b95ac7119d2ecb886/model'

# Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
# loaded_model.predict(pd.DataFrame(input_text))

from transformers import BertTokenizer

# Assuming 'model' is your BertForSequenceClassification model
# and 'tokenizer' is an instance of BertTokenizer or similar compatible with your model

input_text = "SRT190CQ overlap with 190CLCheck engine light on, Connected to Cummins and inspected codes, Fault code active for turbo speed sensor, Opened eds to troubleshoot, Inspected if any other fault code was triggering turbo speed code, Inspected sensor supply voltage, voltage was in spec, Inspect intake manifold pressure sensor for carbon build up, removed sensor to inspect, no build up found, Inspect for signal wire open circuit, Removed harness from ECM and ohmed signal to signal from sensor connector to ecm connector, That was less than 10 ohms, Inspected return for a open circuit, ohmed from sensor connector to ecm connector and that was also less than 10 ohms, Checked for turbo sensor shorted to ground, It was greater than 100k ohms, Inspected signal pin and ohmed out signal pin at ecm to inspect for a pin to pin short, All other pins on ecm connecter are not shorted to signal pin for turbo speed sensor connector, Checked ecm calibration history, Update is available but not relavent to active code, Advised replace turbo speed senor, Speed sensor ordered, Replaced speed sensor"

# Tokenize the input
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    return_tensors="pt"
)

# Move the inputs to the same device as the model
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Get predictions
with torch.no_grad():
    outputs = model(**inputs)

# Check if outputs is a tuple (for older versions of transformers)
if isinstance(outputs, tuple):
    logits = outputs[0]
else:
    logits = outputs.logits

# Assuming you want the class with the highest score
predictions = torch.argmax(logits, dim=-1)

# Display predictions
display(predictions)

# Load the saved model artifacts 
# artifact_path = os.path.join(logged_model, "data")  
# print("label_encoder not found")  
# label_encoder = le

# Load the LabelEncoder object from the artifacts  
artifacts_uri = mlflow.get_artifact_uri()
label_encoder_path = os.path.join(model_dir, "data", "config.pkl") #os.path.join(artifacts_uri.replace("dbfs:/", "/dbfs/"), "data", "config.pkl")  
# print(label_encoder_path)
with open(label_encoder_path, "rb") as f:  
    config = pickle.load(f)  
label_encoder = config["label_encoder"]
display(label_encoder.inverse_transform(predictions.cpu())[0])

tensor([6])

'ACT(E) CONSEQUENTIAL DAMAGE'

# Model Inferencing

## AI Query Agent

In [0]:
# %sql
SELECT ai_query('endpoint_google_bert_base_uncased_fine_tuned_20k_v1',
    request => "SRT190CQ overlap with 190CLCheck engine light on, Connected to Cummins and inspected codes, Fault code active for turbo speed sensor, Opened eds to troubleshoot, Inspected if any other fault code was triggering turbo speed code, Inspected sensor supply voltage, voltage was in spec, Inspect intake manifold pressure sensor for carbon build up, removed sensor to inspect, no build up found, Inspect for signal wire open circuit, Removed harness from ECM and ohmed signal to signal from sensor connector to ecm connector, That was less than 10 ohms, Inspected return for a open circuit, ohmed from sensor connector to ecm connector and that was also less than 10 ohms, Checked for turbo sensor shorted to ground, It was greater than 100k ohms, Inspected signal pin and ohmed out signal pin at ecm to inspect for a pin to pin short, All other pins on ecm connecter are not shorted to signal pin for turbo speed sensor connector, Checked ecm calibration history, Update is available but not relavent to active code, Advised replace turbo speed senor, Speed sensor ordered, 
    Replaced speed sensor")

## Reference Code for querying the endpoint outside of Databricks

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    url = 'https://adb-7018564609060221.1.azuredatabricks.net/serving-endpoints/endpoint_google_bert_base_uncased_fine_tuned_20k_v1/invocations'
    headers = {'Authorization': f'Bearer {os.environ.get("DATABRICKS_TOKEN")}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

In [0]:
query = f"select * from cmidev.cbu_ccims_poc.ccims_claim_inspection_feature where FAILURE_MODE_BUCKET != 'Not Inspected' AND CLAIM_ID_SEQ  = '21469868'"

test_claims =spark.sql(query)

final_test_df = ConcatenateFields(test_claims)
docs = final_test_df.select('DOCUMENT').toPandas()['DOCUMENT']
# test_claims_list = [str(doc).lower() for doc in docs]

# print(test_claims_list)
prediction = score_model(docs)
print(prediction)

# Conclusion

Pre-trained Bert-Base-Uncased model from Hugging Face on our custom training data provided accuracy of 9.1%. This model is successfully registered on MLFlow and can be accessed via below end point:

https://adb-7018564609060221.1.azuredatabricks.net/serving-endpoints/endpoint_google_bert_base_uncased_fine_tuned_20k_v1/invocations
